In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
import json

In [2]:
start_url = "https://boardgamegeek.com/browse/boardgame"
response = requests.get(start_url)
soup_first_page = BeautifulSoup(response.text, "html.parser")

In [3]:
def get_links_one_page(soup):
    links = []
    for link in soup.find_all("a"):
        if link.get("href") is not None:
            if link.get("href").startswith("/boardgame"):
                links.append("https://boardgamegeek.com" + link.get("href"))
    return links[::2]

In [4]:
def get_info_from_first_page(soup, dic):
    ranks = soup.find_all("td", class_="collection_rank")
    ranks = [rank.text for rank in ranks]
    ranks = [rank.replace("\n", "") for rank in ranks]
    ranks = [rank.replace("\t", "") for rank in ranks]
    ranks = [int(rank) for rank in ranks]
    dic["rank"].extend(ranks)
    titels = soup.find_all("a", class_="primary")
    titels = [titel.text for titel in titels]
    dic["name"].extend(titels)
    years = soup.find_all("span", class_="smallerfont")
    years = [year.text for year in years]
    years = [year.replace("(", "").replace(")", "") for year in years]
    years = [int(year) for year in years]
    dic["year"].extend(years)
    ratings = soup.find_all("td", class_="collection_bggrating")
    ratings = soup.find_all("td", class_="collection_bggrating")
    ratings = [rating.text for rating in ratings]
    ratings = [rating.replace("\n", "") for rating in ratings]
    ratings = [rating.replace("\t", "") for rating in ratings]
    ratings = [float(rating) for rating in ratings]
    for i,rate in enumerate(ratings):
        if i%3 == 1:
            dic["rating"].append(rate)
        if i%3 == 2:
            dic["num_voters"].append(rate)

    dic["num_voters"] = [int(voters) for voters in dic["num_voters"]]

In [5]:
def get_info_sub_page(link, dic):

    id = link.split("/")[-2]
    id  = int(id)
    dic["id"].append(id)
    driver = webdriver.Chrome()

    driver.get(link)

    soup = BeautifulSoup(driver.page_source, 'html')
    driver.close()
    ul = soup.find("ul", class_="gameplay")
    
    if ul.find("span", {"ng-if": "min > 0"}) is not None:
        min_players = ul.find("span", {"ng-if": "min > 0"})
        min_players = int(min_players.text)
    else:
        min_players = None
    dic["min_players"].append(min_players)
    if ul.find("span", {"ng-if":"max>0 && min != max"}) is not None:
        max_players = ul.find("span", {"ng-if":"max>0 && min != max"})
        max_players = int( max_players.text[-1])
    else:
        max_players = None
    dic["max_players"].append(max_players)

    if  ul.find("span", {"itemprop":"suggestedMinAge"}) is not None:
        min_age = ul.find("span", {"itemprop":"suggestedMinAge"})
        min_age = int(min_age.text)
    else:
        min_age = None  

    dic["min_age"].append(min_age)
    if  ul.find("span", {"min":"::geekitemctrl.geekitem.data.item.minplaytime"}).find("span", {"ng-if":"min > 0"}) is not None:
        play_time = ul.find("span", {"min":"::geekitemctrl.geekitem.data.item.minplaytime"}).find("span", {"ng-if":"min > 0"})
        play_time = int(play_time.text)
    else:
        play_time = None
    dic["play_time"].append(play_time)
    if ul.find("span", {"ng-class": "{'gameplay-weight-light': geekitemctrl.geekitem.data.item.stats.avgweight <= 3, 'gameplay-weight-medium': geekitemctrl.geekitem.data.item.stats.avgweight > 3,'gameplay-weight-heavy': geekitemctrl.geekitem.data.item.stats.avgweight > 4 }"})is not None:
        complexity = ul.find("span", {"ng-class": "{'gameplay-weight-light': geekitemctrl.geekitem.data.item.stats.avgweight <= 3, 'gameplay-weight-medium': geekitemctrl.geekitem.data.item.stats.avgweight > 3,'gameplay-weight-heavy': geekitemctrl.geekitem.data.item.stats.avgweight > 4 }"})
        complexity = float(complexity.text.replace("\t", "").replace("\n", "").replace(" ", ""))
    else:
        complexity = None
    dic["complexity"].append(complexity)
    if soup.find("span", {"ng-if": "classification.display_inline"}) is not None:
        types = soup.find("span", {"ng-if": "classification.display_inline"})
        types_list = types.find_all("a")
        types_list = [type.text for type in types_list]
    else:
        types_list = []
    dic["categories"].append(types_list)
    print(types_list)

In [9]:
dic_first = {
    "id": [],
    "name": [],
    "year": [],
    "rank": [],
    "rating": [],
    "num_voters": [],
    "play_time": [],
    "min_age": [],
    "max_players": [],
    "min_players": [],
    "complexity": [],
    "owned users": [],
    "mechanics": [],
    "categories": [],
}
dic = dic_first.copy()

In [ ]:
link = "https://boardgamegeek.com/boardgame/224517/brass-birmingham"
response = requests.get(link)
soup= BeautifulSoup(response.text, "html.parser")
scripts = soup.find_all("script")

string = scripts[2].text
string = string.split("GEEK.geekitemPreload = ")[1].split("GEEK.geekitemSettingse")[0].replace("\n", "").replace("\t", "").replace("\n", "")
string = string[:-3] + "}"


{"item":{"itemdata":[{"datatype":"geekitem_fielddata","fieldname":"name","title":"Primary Name","primaryname":true,"required":true,"unclickable":true,"fullcredits":true,"subtype":"boardgame","keyname":"name"},{"datatype":"geekitem_fielddata","fieldname":"alternatename","title":"Alternate Names","alternate":true,"unclickable":true,"fullcredits":true,"subtype":"boardgame","keyname":"alternatename"},{"datatype":"geekitem_fielddata","fieldname":"yearpublished","title":"Year Released","fullcredits":true,"subtype":"boardgame","keyname":"yearpublished"},{"datatype":"geekitem_fielddata","fieldname":"minplayers","title":"Minimum Players","subtype":"boardgame","keyname":"minplayers"},{"datatype":"geekitem_fielddata","fieldname":"maxplayers","title":"Maximum Players","subtype":"boardgame","keyname":"maxplayers"},{"datatype":"geekitem_fielddata","fieldname":"minplaytime","title":"Minimum Playing Time","createposttext":" minutes","posttext":" minutes","subtype":"boardgame","keyname":"minplaytime"},

In [13]:

id = link.split("/")[-2]
id  = int(id)
dic["id"].append(id)
driver = webdriver.Chrome()

driver.get(link)

soup = BeautifulSoup(driver.page_source, 'html')
driver.close()
if soup.find("ul", class_="gameplay") is not None:
    ul = soup.find("ul", class_="gameplay")
else:
    ul = None

if ul.find("span", {"ng-if": "min > 0"}) is not None:
    min_players = ul.find("span", {"ng-if": "min > 0"})
    min_players = int(min_players.text)
else:
    min_players = None
dic["min_players"].append(min_players)
if ul.find("span", {"ng-if":"max>0 && min != max"}) is not None:
    max_players = ul.find("span", {"ng-if":"max>0 && min != max"})
    max_players = int( max_players.text[-1])
else:
    max_players = None
dic["max_players"].append(max_players)

if  ul.find("span", {"itemprop":"suggestedMinAge"}) is not None:
    min_age = ul.find("span", {"itemprop":"suggestedMinAge"})
    min_age = int(min_age.text)
else:
    min_age = None  

dic["min_age"].append(min_age)
if  ul.find("span", {"min":"::geekitemctrl.geekitem.data.item.minplaytime"}).find("span", {"ng-if":"min > 0"}) is not None:
    play_time = ul.find("span", {"min":"::geekitemctrl.geekitem.data.item.minplaytime"}).find("span", {"ng-if":"min > 0"})
    play_time = int(play_time.text)
else:
    play_time = None
dic["play_time"].append(play_time)
if ul.find("span", {"ng-class": "{'gameplay-weight-light': geekitemctrl.geekitem.data.item.stats.avgweight <= 3, 'gameplay-weight-medium': geekitemctrl.geekitem.data.item.stats.avgweight > 3,'gameplay-weight-heavy': geekitemctrl.geekitem.data.item.stats.avgweight > 4 }"})is not None:
    complexity = ul.find("span", {"ng-class": "{'gameplay-weight-light': geekitemctrl.geekitem.data.item.stats.avgweight <= 3, 'gameplay-weight-medium': geekitemctrl.geekitem.data.item.stats.avgweight > 3,'gameplay-weight-heavy': geekitemctrl.geekitem.data.item.stats.avgweight > 4 }"})
    complexity = float(complexity.text.replace("\t", "").replace("\n", "").replace(" ", ""))
else:
    complexity = None
dic["complexity"].append(complexity)
if soup.find("span", {"ng-if": "classification.display_inline"}) is not None:
    types = soup.find("span", {"ng-if": "classification.display_inline"})
    types_list = types.find_all("a")
    types_list = [type.text for type in types_list]
else:
    types_list = []
dic["categories"].append(types_list)
print(types_list)

['Strategy']
